# Análisis Exploratorio de Datos (EDA)

Este notebook contiene el análisis exploratorio completo de los datos almacenados en PostgreSQL.


## 1. Conexión y Carga de Datos

En esta sección establecemos la conexión con la base de datos PostgreSQL y configuramos las bibliotecas necesarias para el análisis.


In [ ]:
# Importar bibliotecas necesarias
import sys
import os

try:
    from ..Utils.path_manager import PathManager
except ImportError:
    # Si falla el import relativo 
    current_dir = os.getcwd()  # Directorio actual de trabajo
    if 'Notebooks' in current_dir:
        avance1_dir = os.path.dirname(current_dir)
    else:
        # Intentar calcular desde la ubicación relativa
        avance1_dir = os.path.abspath(os.path.join(current_dir, '..'))
    
    utils_dir = os.path.join(avance1_dir, 'Utils')
    if utils_dir not in sys.path:
        sys.path.insert(0, utils_dir)
    from path_manager import PathManager

# Configurar sys.path usando PathManager (patrón Singleton)
path_manager = PathManager.get_instance()
path_manager.setup_sys_path()

# Importar DBConnector desde la raíz del proyecto
from Database.db_connector import DBConnector

# Importar bibliotecas de análisis y visualización
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Configurar estilo de visualización
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("✓ Bibliotecas importadas correctamente")
print(f"✓ Project root configurado: {path_manager.get_project_root()}")


In [ ]:
# Obtener el Engine de la base de datos usando DBConnector (patrón Singleton)
db = DBConnector.get_instance()
engine = db.get_engine()

print("✓ Conexión a la base de datos establecida")
print(f"✓ Engine obtenido: {type(engine).__name__}")


### Función para Cargar Tablas

Función genérica para cargar cualquier tabla de la base de datos en un DataFrame de pandas.


In [ ]:
def load_table_to_df(table_name, engine):
    """
    Carga una tabla completa de la base de datos PostgreSQL en un DataFrame de pandas.
    
    Args:
        table_name (str): Nombre de la tabla a cargar
        engine: Engine de SQLAlchemy para la conexión a la base de datos
    
    Returns:
        pd.DataFrame: DataFrame con todos los datos de la tabla
    
    Example:
        >>> df_usuarios = load_table_to_df('usuarios', engine)
    """
    # Una consulta sencilla para cargar la tabla completa
    query = f"SELECT * FROM {table_name};"
    
    return pd.read_sql(query, engine)


## 2. Análisis de Tipos de Datos y Ajustes Necesarios

### Problema Identificado

Cuando se cargan datos desde CSV usando `COPY` sin especificar tipos de datos explícitos en pandas, PostgreSQL puede inferir tipos incorrectos o almacenar datos como `VARCHAR`/`TEXT` cuando deberían ser tipos específicos como `NUMERIC`, `TIMESTAMP` o `INTEGER`.

### Columnas que Requieren Ajuste de Tipos

Basado en el análisis de los modelos ORM y la estructura esperada de la base de datos, las siguientes columnas requieren ajustes obligatorios:


### 2.1. Columnas NUMERIC(10, 2) - Montos y Precios

**Problema**: Los valores monetarios pueden cargarse como `VARCHAR` o `TEXT` si el CSV contiene formatos con símbolos de moneda, comas como separadores de miles, o espacios.

**Tablas afectadas**:
1. **productos.precio** - Precio unitario del producto
2. **ordenes.total** - Monto total de la orden
3. **detalle_ordenes.precio_unitario** - Precio unitario al momento de la orden
4. **ordenes_metodos_pago.monto_pagado** - Monto pagado con cada método
5. **historial_pagos.monto** - Monto del pago


### 2.2. Columnas TIMESTAMP/DateTime - Fechas y Horas

**Problema**: Las fechas pueden cargarse como `VARCHAR` o `TEXT` si el formato en el CSV no coincide exactamente con el formato esperado por PostgreSQL (`YYYY-MM-DD HH:MM:SS`), o si pandas las interpreta como strings.

**Tablas afectadas**:
1. **usuarios.fecha_registro** - Fecha de registro del usuario
2. **ordenes.fecha_orden** - Fecha y hora de creación de la orden
3. **carrito.fecha_agregado** - Fecha en que se agregó el producto al carrito
4. **resenas_productos.fecha** - Fecha de publicación de la reseña
5. **historial_pagos.fecha_pago** - Fecha del pago


### 2.3. Columnas INTEGER - Cantidades y Calificaciones

**Problema**: Los valores numéricos enteros pueden cargarse como `VARCHAR` o `TEXT` si el CSV contiene espacios, caracteres especiales, o si pandas los interpreta como strings.

**Tablas afectadas**:
1. **productos.stock** - Cantidad disponible en inventario
2. **detalle_ordenes.cantidad** - Cantidad de productos en el detalle
3. **carrito.cantidad** - Cantidad del producto en el carrito
4. **resenas_productos.calificacion** - Calificación del producto (1-5)


### 2.4. Columnas ENUM - Estados

**Problema**: Los valores de enum pueden cargarse como `VARCHAR` si no se validan correctamente durante la carga.

**Tablas afectadas**:
1. **ordenes.estado** - Estado de la orden (Pendiente, Enviado, Completado, Cancelado)
2. **historial_pagos.estado_pago** - Estado del pago (Procesando, Pagado, Fallido, Reembolsado)

**Nota**: Los ENUMs se crean correctamente con SQLAlchemy, pero los valores en el CSV deben coincidir exactamente con los valores del enum.


## 3. Scripts SQL para Ajuste de Tipos de Datos

A continuación se presentan los scripts SQL necesarios para ajustar los tipos de datos después de la carga inicial. Estos scripts deben ejecutarse en PostgreSQL después de cargar los datos desde los CSV.


### 3.1. Ajuste de Columnas NUMERIC(10, 2)

**IMPORTANTE**: Antes de ejecutar estos ALTER TABLE, asegúrate de que los datos en las columnas sean convertibles a NUMERIC. Si hay valores con símbolos de moneda ($, €), comas como separadores de miles, o espacios, necesitarás limpiarlos primero.


In [ ]:
-- ============================================================
-- AJUSTE DE TIPOS DE DATOS: COLUMNAS NUMERIC(10, 2)
-- ============================================================
-- Ejecutar estos comandos después de cargar los datos desde CSV
-- para asegurar que los montos y precios tengan el tipo correcto

-- 1. productos.precio
ALTER TABLE productos 
ALTER COLUMN precio TYPE NUMERIC(10, 2) 
USING precio::NUMERIC(10, 2);

-- 2. ordenes.total
ALTER TABLE ordenes 
ALTER COLUMN total TYPE NUMERIC(10, 2) 
USING total::NUMERIC(10, 2);

-- 3. detalle_ordenes.precio_unitario
ALTER TABLE detalle_ordenes 
ALTER COLUMN precio_unitario TYPE NUMERIC(10, 2) 
USING precio_unitario::NUMERIC(10, 2);

-- 4. ordenes_metodos_pago.monto_pagado
ALTER TABLE ordenes_metodos_pago 
ALTER COLUMN monto_pagado TYPE NUMERIC(10, 2) 
USING monto_pagado::NUMERIC(10, 2);

-- 5. historial_pagos.monto
ALTER TABLE historial_pagos 
ALTER COLUMN monto TYPE NUMERIC(10, 2) 
USING monto::NUMERIC(10, 2);


### 3.2. Ajuste de Columnas TIMESTAMP

**IMPORTANTE**: Antes de ejecutar estos ALTER TABLE, asegúrate de que los datos en las columnas de fecha estén en un formato que PostgreSQL pueda interpretar. El formato esperado es `YYYY-MM-DD HH:MM:SS` o variaciones compatibles.


In [ ]:
-- ============================================================
-- AJUSTE DE TIPOS DE DATOS: COLUMNAS TIMESTAMP
-- ============================================================
-- Ejecutar estos comandos después de cargar los datos desde CSV
-- para asegurar que las fechas tengan el tipo TIMESTAMP correcto

-- 1. usuarios.fecha_registro
ALTER TABLE usuarios 
ALTER COLUMN fecha_registro TYPE TIMESTAMP 
USING fecha_registro::TIMESTAMP;

-- 2. ordenes.fecha_orden
ALTER TABLE ordenes 
ALTER COLUMN fecha_orden TYPE TIMESTAMP 
USING fecha_orden::TIMESTAMP;

-- 3. carrito.fecha_agregado
ALTER TABLE carrito 
ALTER COLUMN fecha_agregado TYPE TIMESTAMP 
USING fecha_agregado::TIMESTAMP;

-- 4. resenas_productos.fecha
ALTER TABLE resenas_productos 
ALTER COLUMN fecha TYPE TIMESTAMP 
USING fecha::TIMESTAMP;

-- 5. historial_pagos.fecha_pago
ALTER TABLE historial_pagos 
ALTER COLUMN fecha_pago TYPE TIMESTAMP 
USING fecha_pago::TIMESTAMP;


### 3.3. Ajuste de Columnas INTEGER

**IMPORTANTE**: Antes de ejecutar estos ALTER TABLE, asegúrate de que los datos sean números enteros válidos sin decimales, espacios o caracteres especiales.


In [ ]:
-- ============================================================
-- AJUSTE DE TIPOS DE DATOS: COLUMNAS INTEGER
-- ============================================================
-- Ejecutar estos comandos después de cargar los datos desde CSV
-- para asegurar que las cantidades y calificaciones tengan el tipo INTEGER correcto

-- 1. productos.stock
ALTER TABLE productos 
ALTER COLUMN stock TYPE INTEGER 
USING stock::INTEGER;

-- 2. detalle_ordenes.cantidad
ALTER TABLE detalle_ordenes 
ALTER COLUMN cantidad TYPE INTEGER 
USING cantidad::INTEGER;

-- 3. carrito.cantidad
ALTER TABLE carrito 
ALTER COLUMN cantidad TYPE INTEGER 
USING cantidad::INTEGER;

-- 4. resenas_productos.calificacion
ALTER TABLE resenas_productos 
ALTER COLUMN calificacion TYPE INTEGER 
USING calificacion::INTEGER;


### 3.4. Validación de Columnas ENUM

**IMPORTANTE**: Los ENUMs se crean correctamente con SQLAlchemy, pero es importante validar que los valores en el CSV coincidan exactamente con los valores permitidos del enum. Si hay discrepancias, necesitarás limpiar los datos antes de la carga o usar una función de conversión.

**Valores permitidos para `estado_orden`**:
- 'Pendiente'
- 'Enviado'
- 'Completado'
- 'Cancelado'

**Valores permitidos para `estado_pago`**:
- 'Procesando'
- 'Pagado'
- 'Fallido'
- 'Reembolsado'


In [ ]:
-- ============================================================
-- VALIDACIÓN Y LIMPIEZA DE COLUMNAS ENUM
-- ============================================================
-- Verificar valores únicos en las columnas enum antes de la conversión

-- 1. Verificar valores únicos en ordenes.estado
SELECT DISTINCT estado, COUNT(*) as cantidad
FROM ordenes
GROUP BY estado
ORDER BY cantidad DESC;

-- 2. Verificar valores únicos en historial_pagos.estado_pago
SELECT DISTINCT estado_pago, COUNT(*) as cantidad
FROM historial_pagos
GROUP BY estado_pago
ORDER BY cantidad DESC;

-- Si encuentras valores que no coinciden con el enum, necesitarás limpiarlos:
-- Ejemplo para ordenes.estado (ajustar según los valores encontrados):
-- UPDATE ordenes 
-- SET estado = 'Pendiente' 
-- WHERE estado NOT IN ('Pendiente', 'Enviado', 'Completado', 'Cancelado');

-- Ejemplo para historial_pagos.estado_pago (ajustar según los valores encontrados):
-- UPDATE historial_pagos 
-- SET estado_pago = 'Procesando' 
-- WHERE estado_pago NOT IN ('Procesando', 'Pagado', 'Fallido', 'Reembolsado');


### 3.5. Script Completo de Ajuste de Tipos

A continuación se presenta un script SQL completo que ejecuta todos los ajustes de tipos de datos en el orden correcto:


In [ ]:
-- ============================================================
-- SCRIPT COMPLETO: AJUSTE DE TIPOS DE DATOS
-- ============================================================
-- Ejecutar este script completo después de cargar los datos desde CSV
-- para asegurar la integridad semántica de los tipos de datos

BEGIN;

-- ==================== COLUMNAS NUMERIC(10, 2) ====================
ALTER TABLE productos 
ALTER COLUMN precio TYPE NUMERIC(10, 2) 
USING precio::NUMERIC(10, 2);

ALTER TABLE ordenes 
ALTER COLUMN total TYPE NUMERIC(10, 2) 
USING total::NUMERIC(10, 2);

ALTER TABLE detalle_ordenes 
ALTER COLUMN precio_unitario TYPE NUMERIC(10, 2) 
USING precio_unitario::NUMERIC(10, 2);

ALTER TABLE ordenes_metodos_pago 
ALTER COLUMN monto_pagado TYPE NUMERIC(10, 2) 
USING monto_pagado::NUMERIC(10, 2);

ALTER TABLE historial_pagos 
ALTER COLUMN monto TYPE NUMERIC(10, 2) 
USING monto::NUMERIC(10, 2);

-- ==================== COLUMNAS TIMESTAMP ====================
ALTER TABLE usuarios 
ALTER COLUMN fecha_registro TYPE TIMESTAMP 
USING fecha_registro::TIMESTAMP;

ALTER TABLE ordenes 
ALTER COLUMN fecha_orden TYPE TIMESTAMP 
USING fecha_orden::TIMESTAMP;

ALTER TABLE carrito 
ALTER COLUMN fecha_agregado TYPE TIMESTAMP 
USING fecha_agregado::TIMESTAMP;

ALTER TABLE resenas_productos 
ALTER COLUMN fecha TYPE TIMESTAMP 
USING fecha::TIMESTAMP;

ALTER TABLE historial_pagos 
ALTER COLUMN fecha_pago TYPE TIMESTAMP 
USING fecha_pago::TIMESTAMP;

-- ==================== COLUMNAS INTEGER ====================
ALTER TABLE productos 
ALTER COLUMN stock TYPE INTEGER 
USING stock::INTEGER;

ALTER TABLE detalle_ordenes 
ALTER COLUMN cantidad TYPE INTEGER 
USING cantidad::INTEGER;

ALTER TABLE carrito 
ALTER COLUMN cantidad TYPE INTEGER 
USING cantidad::INTEGER;

ALTER TABLE resenas_productos 
ALTER COLUMN calificacion TYPE INTEGER 
USING calificacion::INTEGER;

COMMIT;

-- ==================== VERIFICACIÓN ====================
-- Ejecutar estas consultas para verificar que los tipos se ajustaron correctamente

SELECT 
    table_name, 
    column_name, 
    data_type, 
    numeric_precision, 
    numeric_scale
FROM information_schema.columns
WHERE table_schema = 'public'
    AND column_name IN (
        'precio', 'total', 'precio_unitario', 'monto_pagado', 'monto',
        'fecha_registro', 'fecha_orden', 'fecha_agregado', 'fecha', 'fecha_pago',
        'stock', 'cantidad', 'calificacion'
    )
ORDER BY table_name, column_name;


## 4. Resumen de Ajustes Necesarios

### Lista Completa de Columnas que Requieren Ajuste de Tipos

#### 4.1. Columnas NUMERIC(10, 2) - Total: 5 columnas

| Tabla | Columna | Descripción |
|-------|---------|-------------|
| `productos` | `precio` | Precio unitario del producto |
| `ordenes` | `total` | Monto total de la orden |
| `detalle_ordenes` | `precio_unitario` | Precio unitario al momento de la orden |
| `ordenes_metodos_pago` | `monto_pagado` | Monto pagado con cada método |
| `historial_pagos` | `monto` | Monto del pago |

#### 4.2. Columnas TIMESTAMP - Total: 5 columnas

| Tabla | Columna | Descripción |
|-------|---------|-------------|
| `usuarios` | `fecha_registro` | Fecha de registro del usuario |
| `ordenes` | `fecha_orden` | Fecha y hora de creación de la orden |
| `carrito` | `fecha_agregado` | Fecha en que se agregó el producto al carrito |
| `resenas_productos` | `fecha` | Fecha de publicación de la reseña |
| `historial_pagos` | `fecha_pago` | Fecha del pago |

#### 4.3. Columnas INTEGER - Total: 4 columnas

| Tabla | Columna | Descripción |
|-------|---------|-------------|
| `productos` | `stock` | Cantidad disponible en inventario |
| `detalle_ordenes` | `cantidad` | Cantidad de productos en el detalle |
| `carrito` | `cantidad` | Cantidad del producto en el carrito |
| `resenas_productos` | `calificacion` | Calificación del producto (1-5) |

#### 4.4. Columnas ENUM - Total: 2 columnas (requieren validación)

| Tabla | Columna | Valores Permitidos |
|-------|---------|-------------------|
| `ordenes` | `estado` | Pendiente, Enviado, Completado, Cancelado |
| `historial_pagos` | `estado_pago` | Procesando, Pagado, Fallido, Reembolsado |

### Total de Ajustes Necesarios

- **16 columnas** requieren ajuste de tipo de datos
- **5 columnas** NUMERIC(10, 2)
- **5 columnas** TIMESTAMP
- **4 columnas** INTEGER
- **2 columnas** ENUM (requieren validación de valores)

### Orden de Ejecución Recomendado

1. **Primero**: Validar valores ENUM (sección 3.4)
2. **Segundo**: Ajustar columnas NUMERIC(10, 2) (sección 3.1)
3. **Tercero**: Ajustar columnas TIMESTAMP (sección 3.2)
4. **Cuarto**: Ajustar columnas INTEGER (sección 3.3)
5. **Finalmente**: Ejecutar consulta de verificación (sección 3.5)

### Notas Importantes

1. **Antes de ejecutar los ALTER TABLE**, verifica que los datos sean convertibles al tipo deseado
2. **Para NUMERIC**: Asegúrate de que no haya símbolos de moneda ($, €), comas como separadores de miles, o espacios
3. **Para TIMESTAMP**: Asegúrate de que las fechas estén en formato `YYYY-MM-DD HH:MM:SS` o compatible
4. **Para INTEGER**: Asegúrate de que no haya decimales, espacios o caracteres especiales
5. **Para ENUM**: Valida que todos los valores coincidan exactamente con los valores permitidos del enum
6. **Usa transacciones**: El script completo (sección 3.5) usa `BEGIN` y `COMMIT` para asegurar que todos los cambios se apliquen o ninguno
